In [1]:
from dotenv import load_dotenv
import os

from typing import TypedDict, Annotated
from langgraph.checkpoint.memory import MemorySaver
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage, SystemMessage, ToolMessage
from langgraph.prebuilt import ToolNode, InjectedState
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.types import Command
from langchain_core.tools import tool, InjectedToolCallId


from gen_test_cases import generate_test_cases_from_requirements
from gen_requirements import generate_requirement_from_doc 

load_dotenv()
llm_api_key = os.getenv("GOOGLE_API_KEY")

In [2]:

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0.3,
    max_tokens = None,
    api_key = llm_api_key,
)

In [3]:
class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], add_messages]
    project_name: str
    context: str
    requirements: list
    testCases: list


In [ ]:
@tool
def generate_requirements_from_document_pdf_tool(
    x: Annotated[str, "base64 encoded string representation of a pdf file"],
    tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """
    Extracts a brief summary and a list of requirements from a project document.

    **Inputs:**
    - `x` (str): A **base64-encoded** string representation of a PDF file containing project details.
        *This string may be very long (up to 8000 tokens) and does not contain human-readable text.
        Pass it exactly as provided—do not modify, interpret, or truncate it.*
    
    **Behavior:**
    - Decodes the PDF document from the base64 input.
    - Analyzes the document content to extract key **project requirements**.
    - Generates a **concise summary** of the document.

    **Outputs:**
    - Updates `state["context"]` with the extracted summary.
    - Updates `state["requirements"]` with a structured list of requirements.
    - Adds a success message in `state["messages"]`.

    **Error Handling:**
    - If the PDF document is **invalid or cannot be processed**, returns an error message.
    """
    requirements, context = generate_requirement_from_doc(x)
    return Command(update={
        "context": context,
        "requirements": requirements,
        "messages": [
            ToolMessage(
                "Successfully created requirements from the document",
                tool_call_id=tool_call_id
            )
        ]
    })


In [ ]:
@tool
def generate_testCases_fromRequirements_tool(
    state: Annotated[AgentState, InjectedState],
    tool_call_id: Annotated[str, InjectedToolCallId]
) -> Command:
    """
    Generates a list of test cases from provided requirements.

    **Inputs:**
    - `state["requirements"]` (list[str]): A list of software or system requirements.
    - `state["context"]` (str): A brief summary to provide additional context.

    **Behavior:**
    - If `requirements` or `context` are missing, returns an error message.
    - Processes requirements to generate structured test cases.
    - Returns a list of test cases in `state["testCases"]`.

    **Outputs:**
    - Updates `state["testCases"]` with generated test cases.
    - Adds a success message to `state["messages"]`.
    """
    requirements = state.get('requirements')
    context = state.get('context')
    if not requirements or not context:
        return Command(update={
            "messages": [
            ToolMessage(
                "Requirements or Context missing",
                tool_call_id=tool_call_id
            )
        ]
    })
    testCases_list = generate_test_cases_from_requirements(requirements, context)
    return Command(update={
        "testCases": testCases_list,
        "messages": [
            ToolMessage(
                "Successfully created test cases from the requirements",
                tool_call_id=tool_call_id
            )
        ]
    })

In [ ]:

tools = [generate_testCases_fromRequirements_tool,
         generate_requirements_from_document_pdf_tool]
llm_with_tools = llm.bind_tools(tools)
tools_node = ToolNode(tools)

In [ ]:

def call_model(state: AgentState):
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": response}

In [8]:
def custom_tool_condition(state: AgentState):
    messages = state["messages"]
    last_message = messages[-1]
    if isinstance(last_message, AIMessage) and last_message.tool_calls:
        return "tools_node"
    return END

In [ ]:
builder = StateGraph(AgentState)
builder.add_node("call_model", call_model)
builder.add_node("tools_node", tools_node)

builder.add_edge(START,"call_model")
builder.add_conditional_edges(
    "call_model",
    custom_tool_condition,
)
builder.add_edge("tools_node","call_model")
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

In [10]:
#Initialize state by loading a project and its info (if any)
currState = AgentState(messages=[SystemMessage(content="You are a helpful assistant tasked with performing QA in SDLC.")],
                       project_name="QA_automation")

In [19]:
# Get user Input
import base64
with open('../ReqView-Example_Software_Requirements_Specification_SRS_Document-1-6.pdf',"rb") as f:
    pdf_data = base64.b64encode(f.read()).decode('utf-8')

userInputContent = "I want you to examine all the docuent. This is base64 string representaiton of the pdf. You should parse it into one of the tools"

# Actually run
currState = {"messages":[(HumanMessage(content=userInputContent))]}
config = {"configurable": {"thread_id": "1"}}
graph.invoke(currState,config)


Call model print:  content='' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'MALFORMED_FUNCTION_CALL', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []} id='run--c0a07472-8d6c-4147-932d-04af47f1d47b-0' usage_metadata={'input_tokens': 403430, 'output_tokens': 0, 'total_tokens': 403430, 'input_token_details': {'cache_read': 0}}


{'messages': [HumanMessage(content=['...', 'This is a string representing project document pdf. Pls extract project requirement from it'], additional_kwargs={}, response_metadata={}, id='6c004b69-0a69-461d-8603-9d426958b5fd'),
  AIMessage(content='```json\n{\n "context": "The PDF document contains information about project requirements, but I am unable to extract a summary without knowing the specifics of the project. Can you provide more information or a specific section of the document to focus on?",\n "requirements": []\n}\n```', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--66967011-394d-4f98-b319-56631be7e1f7-0', usage_metadata={'input_tokens': 403340, 'output_tokens': 59, 'total_tokens': 403399, 'input_token_details': {'cache_read': 0}}),
  HumanMessage(content='I want you to examine all the docuent. This is base64 string representait

In [23]:
graph.get_state({"configurable": {"thread_id": "1"}}).values.get("messages","")

[HumanMessage(content=['...', 'This is a string representing project document pdf. Pls extract project requirement from it'], additional_kwargs={}, response_metadata={}, id='6c004b69-0a69-461d-8603-9d426958b5fd'),
 AIMessage(content='```json\n{\n "context": "The PDF document contains information about project requirements, but I am unable to extract a summary without knowing the specifics of the project. Can you provide more information or a specific section of the document to focus on?",\n "requirements": []\n}\n```', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': []}, id='run--66967011-394d-4f98-b319-56631be7e1f7-0', usage_metadata={'input_tokens': 403340, 'output_tokens': 59, 'total_tokens': 403399, 'input_token_details': {'cache_read': 0}}),
 HumanMessage(content='I want you to examine all the docuent. This is base64 string representaiton of the pdf. 

In [24]:
with open('../ReqView-Example_Software_Requirements_Specification_SRS_Document-1-6.pdf',"rb") as f:
    pdf_data = base64.b64encode(f.read()).decode('utf-8')

requirements1, context1 = generate_requirement_from_doc(pdf_data)

APIConnectionError: Connection error.